## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
dropped_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
dropped_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
apcount = dropped_df["APPLICATION_TYPE"].value_counts()
apcount

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = list(apcount[apcount < 100])

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classcount = dropped_df["CLASSIFICATION"].value_counts()

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classcount.loc[classcount > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classcount[classcount < 700])
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
new_df = pd.get_dummies(dropped_df)

In [11]:
# Split our preprocessed data into our features and target arrays
y = new_df["IS_SUCCESSFUL"].values
X = new_df.drop("IS_SUCCESSFUL", axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state= 42)


In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_feat = len(X_train[0])
hidden_layer1 = 2
hidden_layer2 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1, input_dim=num_input_feat, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 2)                 234       
                                                                 
 dense_4 (Dense)             (None, 4)                 12        
                                                                 
 dense_5 (Dense)             (None, 1)                 5         
                                                                 
Total params: 251
Trainable params: 251
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=150)

Epoch 1/150
804/804 [==============================] - 1s 924us/step - loss: 0.5641 - accuracy: 0.7223
Epoch 2/150
804/804 [==============================] - 1s 886us/step - loss: 0.5639 - accuracy: 0.7228
Epoch 3/150
804/804 [==============================] - 1s 833us/step - loss: 0.5639 - accuracy: 0.7225
Epoch 4/150
804/804 [==============================] - 1s 892us/step - loss: 0.5638 - accuracy: 0.7227
Epoch 5/150
804/804 [==============================] - 1s 803us/step - loss: 0.5640 - accuracy: 0.7221
Epoch 6/150
804/804 [==============================] - 1s 783us/step - loss: 0.5637 - accuracy: 0.7225
Epoch 7/150
804/804 [==============================] - 1s 834us/step - loss: 0.5641 - accuracy: 0.7220
Epoch 8/150
804/804 [==============================] - 1s 831us/step - loss: 0.5636 - accuracy: 0.7222
Epoch 9/150
804/804 [==============================] - 1s 880us/step - loss: 0.5639 - accuracy: 0.7214
Epoch 10/150
804/804 [==============================] - 1s 864us/step - l

804/804 [==============================] - 1s 846us/step - loss: 0.5637 - accuracy: 0.7218
Epoch 80/150
804/804 [==============================] - 1s 849us/step - loss: 0.5636 - accuracy: 0.7228
Epoch 81/150
804/804 [==============================] - 1s 845us/step - loss: 0.5636 - accuracy: 0.7214
Epoch 82/150
804/804 [==============================] - 1s 833us/step - loss: 0.5636 - accuracy: 0.7226
Epoch 83/150
804/804 [==============================] - 1s 833us/step - loss: 0.5636 - accuracy: 0.7229
Epoch 84/150
804/804 [==============================] - 1s 810us/step - loss: 0.5637 - accuracy: 0.7220
Epoch 85/150
804/804 [==============================] - 1s 818us/step - loss: 0.5635 - accuracy: 0.7225
Epoch 86/150
804/804 [==============================] - 1s 820us/step - loss: 0.5636 - accuracy: 0.7219
Epoch 87/150
804/804 [==============================] - 1s 842us/step - loss: 0.5636 - accuracy: 0.7228
Epoch 88/150
804/804 [==============================] - 1s 824us/step - loss:

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5732 - accuracy: 0.7191 - 531ms/epoch - 2ms/step
Loss: 0.5732417702674866, Accuracy: 0.7190670371055603


In [16]:
# Export our model to HDF5 file
# nn.save("AlphabetSoupCharity_Optimization.h5")